<a href="https://colab.research.google.com/github/dickylcd/Financial_Analysis/blob/main/VALE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# Load the historical stock price data - connect from Yahoo Finace API
!pip install yfinance


In [15]:

import yfinance
from datetime import datetime

# Define the dates of data - take 3 months data
year = datetime.today().year
month = datetime.today().month - 3
day = datetime.today().day

start_date = datetime.strptime(f"{year}-{month}-{day}",'%Y-%m-%d').date()
end_date = datetime.today()

# Define stock symbol
stock_symbol = 'VALE'

stock_data = yfinance.download(stock_symbol, start=start_date, end=end_date)
stock_data.columns


[*********************100%%**********************]  1 of 1 completed


Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [16]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
'''
# Load the historical stock price data - connect from Google Drive
from google.colab import drive
drive.mount('/content/drive')

#file_path = '/mnt/data/NASDAQ_ AVGO.csv'
file_path = '/content/drive/MyDrive/a-AI_In_Action/NYSE_VALE.csv'

# Load the data into a DataFrame
stock_data = pd.read_csv(file_path, parse_dates=['Date'])
stock_data = stock_data[(stock_data['Date'] >= '2024-05-01') & (stock_data['Date'] <= '2024-08-16')]
stock_data.set_index('Date', inplace=True)
'''

# Calculate moving averages
stock_data['MA20'] = stock_data['Close'].rolling(window=20).mean()
stock_data['MA50'] = stock_data['Close'].rolling(window=50).mean()

# Display the first few rows to understand the structure of the data
stock_data.head()


,Open,High,Low,Close,Adj Close,Volume,MA20,MA50
Date,,,,,,,,
2024-05-16,12.68,12.69,12.55,12.66,12.208293,18661100,NaN,NaN
2024-05-17,12.74,13.01,12.70,12.99,12.526519,26194700,NaN,NaN
2024-05-20,12.90,13.06,12.81,12.92,12.459017,22794100,NaN,NaN
2024-05-21,13.05,13.14,12.87,12.88,12.420444,27331300,NaN,NaN
2024-05-22,12.81,12.89,12.60,12.65,12.198650,21328100,NaN,NaN


In [17]:
# Generate signals
stock_data['Signal'] = 0
stock_data['Signal'][20:] = np.where(stock_data['MA20'][20:] > stock_data['MA50'][20:], 1, 0)  # Buy signal
stock_data['Position'] = stock_data['Signal'].diff()

# Calculate returns
stock_data['Market Return'] = stock_data['Close'].pct_change()
stock_data['Strategy Return'] = stock_data['Market Return'] * stock_data['Signal'].shift(1)

# Performance metrics
total_return = (stock_data['Strategy Return'].cumsum()[-1])
risk_free_rate = 0.01  # assuming 1% risk-free rate
sharpe_ratio = (stock_data['Strategy Return'].mean() - risk_free_rate) / stock_data['Strategy Return'].std()


<ipython-input-17-e97291712a83>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-17-e97291712a83>:11: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-17-e97291712a83>:13: RuntimeWarning:

divide by zero encountered in scalar divide



In [18]:
# Visualization
fig = go.Figure()
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Close'], mode='lines', name='Close Price'))
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['MA20'], mode='lines', name='20-Day MA'))
fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['MA50'], mode='lines', name='50-Day MA'))

# Buy signals
buy_signals = stock_data[stock_data['Position'] == 1]
sell_signals = stock_data[stock_data['Position'] == -1]
fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', name='Buy Signal', marker=dict(color='green', size=10)))
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', name='Sell Signal', marker=dict(color='red', size=10)))

fig.update_layout(title='Momentum Trading Strategy for NYSE_VALE',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  legend_title='Legend')
fig.show()

# Print performance metrics
print(f'Total Return: {total_return:.2%}')
print(f'Sharpe Ratio: {sharpe_ratio:.2f}')

Total Return: 0.00%
Sharpe Ratio: -inf
